<a href="https://colab.research.google.com/github/sa96102/Python_exam/blob/master/LSTM_Novel_nietzsche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import random, sys, io, re

%tensorflow_version 1.x
import tensorflow
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np

TensorFlow 1.x selected.


Using TensorFlow backend.


### Preprocessing

In [3]:
path = '/content/drive/My Drive/Colab Notebooks/Python_exam/Data/nietzsche.txt'
with io.open(path) as f:
    text = f.read().lower()

print('corpus length : ', len(text))

corpus length :  600893


### Compute Character indices

In [4]:
chars = sorted(list(set(text)))
print('total chars : ', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars)) # one-hot encoding으로 처리할 것이기 때문에 인덱스를 저장.

total chars :  57


### Vectorize Sentences

In [5]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences : ', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences :  200285
Vectorization...


### Create Model

In [6]:
print('Build model...')
model = Sequential()
model.add(LSTM(1024, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001))

Build model...







### Define Callback

In [0]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
    # 확률을 조정. 예측한대로만 할 경우, loop에 빠질 경우가 발생할 수도 있음.

def on_epoch_end(epoch, _): # 한 epoch이 끝날 때마다 실행.
    print('\n----- Generating text after Epoch : %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed : "' + sentence + '"')
    sys.stdout.write(generated)
    # 랜덤으로 40개의 글자 seed를 생성. 그리고 모델에게 이를 주고 뒷부분을 만들어내도록 함. 여기서는 아래 for문과 같이 400글자를 예측하도록 함.

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0] # model.predict()_데이터를 인풋으로 받아 예측.
        next_index = sample(preds, 0.5)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

### Training

In [8]:
model.fit(x, y, batch_size=128, epochs=30, callbacks=[print_callback])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/30





200285/200285 [==============================] - 128s 640us/step - loss: 2.2922

----- Generating text after Epoch : 0
----- Generating with seed : "avagant magnanimity after which nothing "
avagant magnanimity after which nothing the estertation gor the must the outhers the mastion be becoure the destion the sparing and gere the graltes the sumpersation wo men the extrensally pleast to the gand not ics mate as the every the most the mast be of the to the eviliow of the every all to and of the suther that the sting the prilition prabe the made to the mast of the and ines and of devertions the erelition the mone a saruther g
Epoch 2/30
200285/200285 [==============================] - 120s 598us/step - loss: 1.7642

----- Generating text after Epoch : 1
----- Generating with seed : "cess, cannot
maintain itself, therefore "
cess, cannot
maintain itself, therefore the scentuali